##Importing Libraries

In [0]:
#General Libraries
import json
import pandas as pd
import numpy as np

#NLTK PRE_PROCESSING Libraries
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
import string


# Image Extraction Libraries
import os
from keras.applications.vgg16 import VGG16
import cv2
from keras import models
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.models import model_from_json

#For Answer Encoding
from sklearn.preprocessing import LabelEncoder

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

##Question Pre-Processing

In [3]:
from google.colab import drive
drive.mount('/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive/


In [0]:
#Loading the Question files

path = "/gdrive/My Drive/Colab Notebooks/VQA_DATASET/trainQuestions.json"
data = json.load(open(path))
train_questions = pd.DataFrame(data["questions"])

In [5]:
# Checking the dimensions
print("Size of Train Questions : " , train_questions.shape)

Size of Train Questions :  (443757, 3)


In [6]:
train_questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443757 entries, 0 to 443756
Data columns (total 3 columns):
image_id       443757 non-null int64
question       443757 non-null object
question_id    443757 non-null int64
dtypes: int64(2), object(1)
memory usage: 10.2+ MB


In [0]:
#Loading the answers file
path = "/gdrive/My Drive/Colab Notebooks/VQA_DATASET/trainAnswers.json"
data = json.load(open(path))
train_answers = pd.DataFrame(data["annotations"])

In [9]:
train_answers.head()

,answer_type,answers,image_id,multiple_choice_answer,question_id,question_type
0,other,"[{'answer': 'net', 'answer_confidence': 'maybe...",458752,net,458752000,what is this
1,other,"[{'answer': 'pitcher', 'answer_confidence': 'y...",458752,pitcher,458752001,what
2,other,"[{'answer': 'orange', 'answer_confidence': 'ye...",458752,orange,458752002,what color is the
3,yes/no,"[{'answer': 'yes', 'answer_confidence': 'yes',...",458752,yes,458752003,is this
4,other,"[{'answer': 'white', 'answer_confidence': 'yes...",262146,white,262146000,what color is the


In [10]:
# Pre Processing the Question
def word_preprocess(question):
  #Converting it into Lower String
  question = question.str.lower()
  
  #Removing punctuations
  question = question.str.replace('[^\w\s]','')
  
  #Removing Stop words
  #stop_words = set(stopwords.words('english'))
  #question = question.apply(lambda x: " ".join(x for x in x.split() if x not in stop_words))
  
  return question
  
train_questions['question'] = word_preprocess(train_questions['question'])
train_questions['question'].head()

0      what is this photo taken looking through
1             what position is this man playing
2               what color is the players shirt
3    is this man a professional baseball player
4                        what color is the snow
Name: question, dtype: object

In [0]:
# Preparing The Training Data
train_data = []
for i in range(len(train_answers)):
            ans = train_answers['multiple_choice_answer'][i]
            question_id = train_answers['question_id'][i]
            #image_path = imdir%(subtype, subtype, train_anno['annotations'][i]['image_id'])
            question = train_questions['question'][i]
            image_id = train_questions['image_id'][i]
            train_data.append({'ques_id': question_id, 'question': question, 'ans': ans,'img_id':image_id})

In [0]:
question_dt = pd.DataFrame(train_data)

##Image Pre-Processing

In [0]:
base_dir = '/gdrive/My Drive/Colab Notebooks/VQA_DATASET/VQA DATA SET/Train_dataset_1'
images = os.listdir(base_dir)

In [0]:
data=[]
for filename in os.listdir(base_dir):
    if filename.endswith("jpg"): 
        # Your code comes here such as 
        #print(filename)
        data.append(filename)

In [0]:
image_df=pd.DataFrame(data,columns=['Image_Path'])

In [0]:
img_id = [int(x[15:-4]) for x in image_df["Image_Path"]]
image_df["img_id"] = img_id

In [20]:
VGG_model = VGG16(weights='imagenet',
                  include_top=True,
                  input_shape=(224,224,3))
VGG_model.layers.pop()
VGG_model.layers.pop()

W0730 16:08:29.834477 139920978573184 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0730 16:08:29.882212 139920978573184 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0730 16:08:29.894203 139920978573184 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0730 16:08:29.958547 139920978573184 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



553467904/553467096 [==============================] - 114s 0us/step


W0730 16:10:25.025671 139920978573184 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0730 16:10:25.027431 139920978573184 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



In [0]:
# Extracts the outputs of the top 8 layers:
for layer in VGG_model.layers[:17]:
  layer.trainable = False
  #print(layer)

In [22]:
VGG_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:

image_df['image_feature']=' '

for i in range(0,len(image_df['Image_Path'])):cv2
  r=image_df['Image_Path'][i]
  x='/gdrive/My Drive/Colab Notebooks/VQA_DATASET/VQA DATA SET/Train_dataset_1/'+r
  imag=cv2.imread(x)
  #image = preprocess_input(imag)
  image = img_to_array(imag)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  features = VGG_model.predict(image)
  image_df['image_feature'][i]=features[0]

##Merging Image Data & Question  Data

In [0]:
train_data= pd.merge(question_dt, image_df, on=['img_id'], how='inner')

##Answer Pre-Processing

In [0]:
le=LabelEncoder()
le.fit(train_data["ans"])

LabelEncoder()

In [0]:
ans = le.transform(train_data["ans"])

In [0]:
train_data['ans']=ans

In [0]:
train_data

,ans,img_id,ques_id,question,image_name,image_feature
0,5218,25,25000,front giraffes,COCO_train2014_000000000025.jpg,"[1.38433e-05, 5.34205e-05, 0.0001280025, 0.000..."
1,1887,25,25001,giraffes common,COCO_train2014_000000000025.jpg,"[1.38433e-05, 5.34205e-05, 0.0001280025, 0.000..."
10,2993,25,25010,ground next giraffe right,COCO_train2014_000000000025.jpg,"[1.38433e-05, 5.34205e-05, 0.0001280025, 0.000..."
100,3363,149,149001,sky clear,COCO_train2014_000000000149.jpg,"[3.93909e-05, 1.6657e-05, 9.6804e-06, 3.1817e-..."
1000,4219,1522,1522001,kind place,COCO_train2014_000000001522.jpg,"[9.9e-09, 3.589e-07, 9.6e-09, 7e-10, 5.8e-09, ..."
10000,3302,15726,15726000,around neck,COCO_train2014_000000015726.jpg,"[0.0001657593, 6.68884e-05, 1.98387e-05, 5.787..."
10001,5640,15726,15726001,man holding something,COCO_train2014_000000015726.jpg,"[0.0001657593, 6.68884e-05, 1.98387e-05, 5.787..."
10002,5640,15726,15726002,anyone photo wearing glasses,COCO_train2014_000000015726.jpg,"[0.0001657593, 6.68884e-05, 1.98387e-05, 5.787..."
10003,974,15733,15733000,color water,COCO_train2014_000000015733.jpg,"[1.17058e-05, 7.17537e-05, 0.0002711012, 0.000..."
10004,5640,15733,15733001,lose surfboard,COCO_train2014_000000015733.jpg,"[1.17058e-05, 7.17537e-05, 0.0002711012, 0.000..."


##Final Dump to Json

In [0]:
train_data.to_json('/gdrive/My Drive/Colab Notebooks/VQA_DATASET/Final_Dataset.json')

In [0]:
#Copy backup_dataset
final_data.to_json('/gdrive/My Drive/Colab Notebooks/VQA_DATASET/Final_Dataset1.json')